#### http://www.kipris.or.kr/khome/main.jsp
#### ("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"머신러닝"+"machine learning")*AD=[20190624~20240624]

#### 로그인 후 환경설정 셋팅 후 검색
#### 환경설정 - 국제출원일자(FD), 우선권주장일자(PRD)
#### 목록 뷰 스타일 - 서지정보 보기

#### boseon169@gmail.com
#### !@1234boseon


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time
import re
import pandas as pd

In [11]:
class Kipris() :
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.m_page_code = ''
        self.df = pd.DataFrame(columns=['status', 'title', 'ap_num', 'application_date', 'applicant','representative',
                            'final_owner', 'priority_date', 'international_filing_date', 'summary'])

    def open(self) :
        # 사이트 열기
        url = 'http://www.kipris.or.kr/khome/main.jsp'

        self.driver.get(url)

        # 처음에 열린 page코드
        page_dict = {idx: self.driver.window_handles[idx] for idx in range(len(self.driver.window_handles))}

        # 메인페이지 코드
        self.m_page_code = page_dict[0]

    def close(self):
        # 브라우저 종료
        self.driver.quit()

    # 로그인
    def login(self) :
        print('\nlogin() start')
        login = self.driver.find_element(By.ID, 'btnLogin')
        login.click()

        print(self.driver.window_handles)

        # 로그인 팝업창 선택
        self.driver.switch_to.window(self.driver.window_handles[1])

        # 로그인 ID
        login_id = self.driver.find_element(By.ID, 'login_id')
        login_id.send_keys('hwooks.choi@gmail.com')

        # 로그인 pw
        login_pw = self.driver.find_element(By.ID, 'login_pw')
        login_pw.send_keys('9697003kK*')

        # 로그인 버튼
        btn_login = self.driver.find_element(By.ID, 'btn_login')
        btn_login.click()

        # 5초 기다린 후 팝업 닫기
        time.sleep(5)
        self.pop_close()

        print('login() end')

    # 팝업닫기
    def pop_close(self) :
        print('\npop_close() start')

        # 팝업이 있는지 10초동안 확인
        wait = WebDriverWait(self.driver, 10)
        wait.until(lambda d: len(self.driver.window_handles) > 1)

        print('페이지 갯수 : ' , len(self.driver.window_handles))

        self.driver.switch_to.window(self.m_page_code)

        # 팝업이 있을 경우 팝업 닫기
        if len(self.driver.window_handles) > 1 :
            total_site = self.driver.window_handles # 열린 사이트 갯수 (메인페이지 + 팝업창)

            print(f'총 페이지 : {total_site}')
            print(f'메인 페이지 코드 : {self.m_page_code}')

            for page in total_site :
                # 열린 사이트들에서 메인 페이지를 제외한 창 닫기)
                if page != self.m_page_code :
                    self.driver.switch_to.window(page) # 현재 선택된 창을 팝업창으로 변경
                    self.driver.close()

            # 팝업창을 다 닫은 후 메인페이지를 다시 선택
            self.driver.switch_to.window(self.m_page_code)
            print('if end')

        print('pop_close() end')

    # 페이지 이동
    def patent_move(self) :
        print('patent_move start')

        # 특허·실용신안 태그 가져오기
        patent_tag = WebDriverWait(self.driver,3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,'#gnb ul.cbp-hrsub-inner.cbp-hrsub-inner.depth2 li:nth-child(1) a'))
            #'//*[@id="gnb"]/li[1]/div/ul/li[1]/a'
        )

        # 클릭하여 페이지 이동
        patent_tag.click()

        # 페이지 이동 잠시 대기
        time.sleep(3)

        # 검색창 입력
        input_tag = self.driver.find_element(By.ID,'queryText')
        input_tag.send_keys('("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"머신러닝"+"machine learning")*AD=[20190624~20240624]')
        input_tag.send_keys(Keys.ENTER)

        time.sleep(3)

        # 리스트 형식 변경 (요약 함께보기)
        btnSimple = self.driver.find_element(By.CSS_SELECTOR, '#btnTextView')
        btnSimple.click()

        time.sleep(3)

        # 페이지 목록 갯수 설정
        select = Select(self.driver.find_element(By.ID,'opt28'))
        select.select_by_value('90')

        time.sleep(3)

        # 설정된 페이지 목록 갯수로 다시 조회
        select_a = self.driver.find_element(By.CSS_SELECTOR,'#pageSel a')
        select_a.click()

        print('patent_move end')

    # 페이지 목록 이동
    def page_num_move(self) :
        # 총 페이지 수 가져오기
        articles_info = self.driver.find_element(By.CSS_SELECTOR, 'p.articles').text
        total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())
        # print(f"Total pages: {total_pages}")
        #for page_number in range(1, totla_pages):
        for page_number in range(1, 13):
            # 페이지 번호를 클릭하여 페이지 변경
            if page_number != 1 and page_number % 10 != 0:
                page_link = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//span[@class='board_pager03']//a[text()='{page_number}']"))
                )
                page_link.click()
                time.sleep(7)  # 페이지가 로드될 때까지 대기
            elif page_number % 10 == 0:
                page_link = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[10]'))
                )
                page_link.click()
                time.sleep(20)  # 페이지가 로드될 때까지 대기


    # 데이터 셋팅
    def data_setting(self) :
        # 상태 제목 출원번호 출원일자 출원인 대리인 최종권리자 우선권주장일자 국제출원일자 요약
        # 상태 - status
        # 제목 - title
        # 출원번호 - application_number
        # 출원일자 - application_date
        # 출원인 - applicant
        # 대리인 - representative
        # 최종권리자 - final_owner
        # 우선권주장일자 - priority_date
        # 국제출원일자 - international_filing_date
        # 요약 - summary

        print('\ndata_setting start')

        data = self.driver.execute_script("""
            var main_section = document.querySelector('#divMainArticle section.search_section');
            var divs = main_section.querySelectorAll('[id*="divViewSel"]');

            var status_list = []                    //  상태
            var title_list = []                     //  제목
            var ap_num_list = []                    //  출원번호
            var application_date_list = []          //  출원일자
            var applicant_list = []                 //  출원인
            var representative_list = []            //  대리인
            var final_owner_list = []               //  최종권리자
            var priority_date_list = []             //  우선권주장일자
            var international_filing_date_list = [] //  국제출원일자
            var summary_list = []                   //  요약

            for(i=0; i<divs.length; i++){
                div = divs[i];

                //  타이틀
                var h1_tag = div.querySelector('.stitle');

                //  등록상태
                var statusEl = h1_tag.querySelector('#iconStatus');
                var status = statusEl ? statusEl.textContent: '';
                status_list.push(status);

                //  제목
                var titleEl = h1_tag.querySelector('a:nth-of-type(2)');
                var title = titleEl ? titleEl.textContent : '';
                title_list.push(title);

                //  출원번호 출원일자 출원인 대리인 최종권리자 DIV
                var divRealContent = div.querySelector('#divRealContent' + i);

                //  출원번호
                var ap_num_el = divRealContent.querySelector('li:nth-child(2) span.point01');
                var ap_num = ap_num_el ? ap_num_el.textContent : '';
                ap_num_list.push(ap_num);

                //  출원일자
                var ap_date_el = divRealContent.querySelector('li:nth-child(3)');
                var ap_date = ap_date_el ? ap_date_el.textContent.replace(/[^\d]/g,'') : '';    // 출원일자 : 2020.09.25 -> 숫자만 추출
                application_date_list.push(ap_date);

                //  출원인
                var applicant_el = divRealContent.querySelector('ul.search_info_list li:nth-child(4) a');
                var applicant = applicant_el ? applicant_el.textContent : '';
                applicant_list.push(applicant);

                //  대리인
                var representative_el= divRealContent.querySelector('ul.search_info_list li:nth-child(10) a');
                var representative = representative_el ? representative_el.textContent : '';
                representative_list.push(representative);

                //  최종권리자
                var final_owner_el = divRealContent.querySelector('ul.search_info_list li:nth-child(11) a');
                var final_owner = final_owner_el ? final_owner_el.textContent : '';
                final_owner_list.push(final_owner);

                //  우선권주장일자 국제출원일자 DIV
                var div_search_detail = divRealContent.querySelector('#div_search_detail_' + ap_num);

                //  우선권주장일자
                var priority_date_el = div_search_detail.querySelector('ul li:nth-child(1)');
                var priority_date = priority_date_el ? priority_date_el.textContent.replace(/[^\d]/g,'') : '';
                priority_date_list.push(priority_date);

                //  국제출원일자
                var international_filing_date_el = div_search_detail.querySelector('ul li:nth-child(2)');
                var international_filing_date = international_filing_date_el ? international_filing_date_el.textContent.replace(/[^\d]/g,'') : '';
                international_filing_date_list.push(international_filing_date);

                var summary_el = div_search_detail.querySelector('div.search_txt');
                var summary = summary_el ? summary_el.textContent.replace(/[\\n\\t]+/g,'') : '';
                summary_list.push(summary);
            }

            return {
                'status'                    :   status_list,
                'title'                     :   title_list,
                'ap_num'                    :   ap_num_list,
                'application_date'          :   application_date_list,
                'applicant'                 :   applicant_list,
                'representative'            :   representative_list,
                'final_owner'               :   final_owner_list,
                'priority_date'             :   priority_date_list,
                'international_filing_date' :   international_filing_date_list,
                'summary' :   summary_list
            };

        """)

        # 데이터프레임 셋팅
        df = pd.DataFrame({
            'status':data['status'],
            'title':data['title'],
            'ap_num':data['ap_num'],
            'application_date':data['application_date'],
            'applicant':data['applicant'],
            'representative':data['representative'],
            'final_owner':data['final_owner'],
            'priority_date':data['priority_date'],
            'international_filing_date':data['international_filing_date'],
            'summary':data['summary']
            })
        print('data_setting end\n')
        return df



In [12]:
kipris = Kipris()

# 페이지 오픈
kipris.open()

# 팝업 닫기
kipris.pop_close()

# 로그인 진행
kipris.login()

# 페이지 이동
kipris.patent_move()

time.sleep(10)

kipris.data_setting()

time.sleep(3)

articles_info = kipris.driver.find_element(By.CSS_SELECTOR, 'p.articles').text
total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip()) # 총 페이지
current_page = kipris.driver.find_element(By.CSS_SELECTOR, '#divMainArticle div.float_left span.current').text # 현재 페이지

# 처음 1페이지 데이터 저장
kipris.df = pd.concat([kipris.df, kipris.data_setting()], ignore_index=True)
kipris.df.to_csv('KIPRIS_1page.csv', encoding='utf-8-sig',index=False)
# 1페이지부터 마지막 페이지까지 데이터 저장
for i in range(1,total_pages) :
    # 페이지 이동해서 2페이지 부터 저장
    kipris.page_num_move()
    time.sleep(3)
    if current_page != total_pages :
        kipris.df = pd.concat([kipris.df, kipris.data_setting()], ignore_index=True)

#df = pd.DataFrame(columns=['status', 'title', 'ap_num', 'application_date', 'applicant','representative',
#                             'final_owner', 'priority_date', 'international_filing_date', 'summary'])
#df = pd.concat([df, kipris.data_setting()], ignore_index=True)
kipris.df.to_csv('KIPRIS_all.csv', encoding='utf-8-sig',index=False)


pop_close() start
페이지 갯수 :  2
총 페이지 : ['84310D26C3AEF36C4B434346C086A500', 'B35C5C9CF668D79DA46610F0BC95D598']
메인 페이지 코드 : 84310D26C3AEF36C4B434346C086A500
if end
pop_close() end

login() start
['84310D26C3AEF36C4B434346C086A500', '441AC04452AB4011A4702493AAE71010']

pop_close() start
페이지 갯수 :  2
총 페이지 : ['84310D26C3AEF36C4B434346C086A500', 'CD924F19F93619E678DB1CFFC55D9D4C']
메인 페이지 코드 : 84310D26C3AEF36C4B434346C086A500
if end
pop_close() end
login() end
patent_move start
patent_move end

data_setting start
data_setting end


data_setting start
data_setting end



TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CEF2EEA2+31554]
	(No symbol) [0x00007FF6CEEA7ED9]
	(No symbol) [0x00007FF6CED6872A]
	(No symbol) [0x00007FF6CEDB8434]
	(No symbol) [0x00007FF6CEDB853C]
	(No symbol) [0x00007FF6CEDFF6A7]
	(No symbol) [0x00007FF6CEDDD06F]
	(No symbol) [0x00007FF6CEDFC977]
	(No symbol) [0x00007FF6CEDDCDD3]
	(No symbol) [0x00007FF6CEDAA33B]
	(No symbol) [0x00007FF6CEDAAED1]
	GetHandleVerifier [0x00007FF6CF238B1D+3217341]
	GetHandleVerifier [0x00007FF6CF285AE3+3532675]
	GetHandleVerifier [0x00007FF6CF27B0E0+3489152]
	GetHandleVerifier [0x00007FF6CEFDE776+750614]
	(No symbol) [0x00007FF6CEEB375F]
	(No symbol) [0x00007FF6CEEAEB14]
	(No symbol) [0x00007FF6CEEAECA2]
	(No symbol) [0x00007FF6CEE9E16F]
	BaseThreadInitThunk [0x00007FF9037D7344+20]
	RtlUserThreadStart [0x00007FF90479CC91+33]


In [7]:
df

NameError: name 'df' is not defined

In [10]:
kipris.df

,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary
0,등록,"갱신 영역 학습을 통한 인공지능 기반의 영상 분석 시스템 및 방법과, 이를 위한 컴...",1020200124561,20200925,정안수,한강특허법인,정안수,,,인공지능(Artificial Intelligence; AI) 기반의 영상 분석 시스...
1,등록,"인공지능(Artificial Intelligence, AI) 알고리즘 수행 방법, ...",1020200165605,20201201,한국과학기술정보연구원,특허법인(유한)케이비케이,한국과학기술정보연구원,,,"개시한 실시예들은 인공지능(Artificial Intelligence, AI) 알고..."
2,등록,"머신러닝 기반 인공지능 모델 학습, 개발, 배포 및 운영 시스템과 이를 이용한 서비...",1020200164624,20201130,주식회사 애자일소다,우광제,주식회사 애자일소다,,,"머신러닝 기반 인공지능 모델 학습, 개발, 배포 및 운영 시스템과 이를 이용한 서비..."
3,취하,인공지능 돌봄 서비스 시스템(An AI Care Service System),1020200071313,20200612,주식회사 에프램,,,,,Capacitive Sensor Array Mat(602) 장치의 감지 Patter...
4,공개,인공지능 학습용 이미지 인식 데이터 품질관리를 활용한 자동화된 머신러닝 시스템(Au...,1020220140941,20221028,(주) 경성테크놀러지,박명흠,,,,본 발명은 인공지능 학습용 이미지 인식 데이터 품질관리를 활용한 자동화된 머신러닝 ...
...,...,...,...,...,...,...,...,...,...,...
85,등록,"텐던진단장치를 이용한 텐던 손상진단을 위한 인공지능 학습방법과 인공지능 학습시스템,...",1020220169250,20221207,한국건설기술연구원,이준서,한국건설기술연구원,,,본 발명은 유도전압 측정에 기반한 텐던진단장치를 이용하여 텐던의 손상을 진단함에 있...
86,거절,지하철 전동차내와 승강장 및 터널의 공기질(IAQ)을 콘트롤하는 인공지능 시스템공기...,1020190173722,20191224,이세라,,,,,"본 출원발명은 지하철 사령실, 전동차내, 승강장 및 터널내의 실내외공기질(IAQ 0..."
87,등록,인공지능을 활용한 자동화 병상 배정 방법(Method for Automated be...,1020220047582,20220418,주식회사 지티솔루션,최학현,주식회사 지티솔루션,,,본 발명은 입원과 퇴원으로 인한 병상관리를 인공지능(AI:artificial int...
88,등록,전파 측정 결과로부터 무인 항공기를 탐지하고 인공지능에 기반하여 무인 항공기를 식별...,1020220076281,20220622,주식회사 휴라,성병기,주식회사 휴라,,,"본 발명은 인공지능(artificial intelligence, AI)에 기반하여 ..."


# 여기까지 실행 코드

In [ ]:
# 상태 제목 출원번호 출원일자 출원인 대리인 최종권리자 우선권주장일자 국제출원일자 요약
# 상태 - status
# 제목 - title
# 출원번호 - application_number
# 출원일자 - application_date
# 출원인 - applicant
# 대리인 - representative
# 최종권리자 - final_owner
# 우선권주장일자 - priority_date
# 국제출원일자 - international_filing_date
# 요약 - summary

data = kipris.driver.execute_script("""
    var main_section = document.querySelector('#divMainArticle section.search_section');
    var divs = main_section.querySelectorAll('[id*="divViewSel"]');

    var status_list = []                    //  상태
    var title_list = []                     //  제목
    var ap_num_list = []                    //  출원번호
    var application_date_list = []          //  출원일자
    var applicant_list = []                 //  출원인
    var representative_list = []            //  대리인
    var final_owner_list = []               //  최종권리자
    var priority_date_list = []             //  우선권주장일자
    var international_filing_date_list = [] //  국제출원일자
    var summary_list = []                   //  요약

    for(i=0; i<divs.length; i++){
        div = divs[i];

        //  타이틀
        var h1_tag = div.querySelector('.stitle');

        //  등록상태
        var statusEl = h1_tag.querySelector('#iconStatus');
        var status = statusEl ? statusEl.textContent: '';
        status_list.push(status);

        //  제목
        var titleEl = h1_tag.querySelector('a:nth-of-type(2)');
        var title = titleEl ? titleEl.textContent : '';
        title_list.push(title);

        //  출원번호 출원일자 출원인 대리인 최종권리자 DIV
        var divRealContent = div.querySelector('#divRealContent' + i);

        //  출원번호
        var ap_num_el = divRealContent.querySelector('li:nth-child(2) span.point01');
        var ap_num = ap_num_el ? ap_num_el.textContent : '';
        ap_num_list.push(ap_num);

        //  출원일자
        var ap_date_el = divRealContent.querySelector('li:nth-child(3)');
        var ap_date = ap_date_el ? ap_date_el.textContent.replace(/[^\d]/g,'') : '';    // 출원일자 : 2020.09.25 -> 숫자만 추출
        application_date_list.push(ap_date);

        //  출원인
        var applicant_el = divRealContent.querySelector('ul.search_info_list li:nth-child(4) a');
        var applicant = applicant_el ? applicant_el.textContent : '';
        applicant_list.push(applicant);

        //  대리인
        var representative_el= divRealContent.querySelector('ul.search_info_list li:nth-child(10) a');
        var representative = representative_el ? representative_el.textContent : '';
        representative_list.push(representative);

        //  최종권리자
        var final_owner_el = divRealContent.querySelector('ul.search_info_list li:nth-child(11) a');
        var final_owner = final_owner_el ? final_owner_el.textContent : '';
        final_owner_list.push(final_owner);

        //  우선권주장일자 국제출원일자 DIV
        var div_search_detail = divRealContent.querySelector('#div_search_detail_' + ap_num);

        //  우선권주장일자
        var priority_date_el = div_search_detail.querySelector('ul li:nth-child(1)');
        var priority_date = priority_date_el ? priority_date_el.textContent.replace(/[^\d]/g,'') : '';
        priority_date_list.push(priority_date);

        //  국제출원일자
        var international_filing_date_el = div_search_detail.querySelector('ul li:nth-child(2)');
        var international_filing_date = international_filing_date_el ? international_filing_date_el.textContent.replace(/[^\d]/g,'') : '';
        international_filing_date_list.push(international_filing_date);

        var summary_el = div_search_detail.querySelector('div.search_txt');
        var summary = summary_el ? summary_el.textContent.replace(/[\\n\\t]+/g,'') : '';
        summary_list.push(summary);
    }

    return {
        'status'                    :   status_list,
        'title'                     :   title_list,
        'ap_num'                    :   ap_num_list,
        'application_date'          :   application_date_list,
        'applicant'                 :   applicant_list,
        'representative'            :   representative_list,
        'final_owner'               :   final_owner_list,
        'priority_date'             :   priority_date_list,
        'international_filing_date' :   international_filing_date_list,
        'summary' :   summary_list
    };

""")

# 데이터프레임 셋팅
df = pd.DataFrame({
    'status':data['status'],
    'title':data['title'],
    'ap_num':data['ap_num'],
    'application_date':data['application_date'],
    'applicant':data['applicant'],
    'representative':data['representative'],
    'final_owner':data['final_owner'],
    'priority_date':data['priority_date'],
    'international_filing_date':data['international_filing_date'],
    'summary':data['summary']
    })

df


,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary
0,등록,"갱신 영역 학습을 통한 인공지능 기반의 영상 분석 시스템 및 방법과, 이를 위한 컴...",1020200124561,20200925,정안수,한강특허법인,정안수,,,인공지능(Artificial Intelligence; AI) 기반의 영상 분석 시스...
1,등록,"인공지능(Artificial Intelligence, AI) 알고리즘 수행 방법, ...",1020200165605,20201201,한국과학기술정보연구원,특허법인(유한)케이비케이,한국과학기술정보연구원,,,"개시한 실시예들은 인공지능(Artificial Intelligence, AI) 알고..."
2,취하,인공지능 돌봄 서비스 시스템(An AI Care Service System),1020200071313,20200612,주식회사 에프램,,,,,Capacitive Sensor Array Mat(602) 장치의 감지 Patter...
3,공개,인공지능 학습용 이미지 인식 데이터 품질관리를 활용한 자동화된 머신러닝 시스템(Au...,1020220140941,20221028,(주) 경성테크놀러지,박명흠,,,,본 발명은 인공지능 학습용 이미지 인식 데이터 품질관리를 활용한 자동화된 머신러닝 ...
4,등록,"머신러닝 기반 인공지능 모델 학습, 개발, 배포 및 운영 시스템과 이를 이용한 서비...",1020200164624,20201130,주식회사 애자일소다,우광제,주식회사 애자일소다,,,"머신러닝 기반 인공지능 모델 학습, 개발, 배포 및 운영 시스템과 이를 이용한 서비..."
...,...,...,...,...,...,...,...,...,...,...
85,등록,"텐던진단장치를 이용한 텐던 손상진단을 위한 인공지능 학습방법과 인공지능 학습시스템,...",1020220169250,20221207,한국건설기술연구원,이준서,한국건설기술연구원,,,본 발명은 유도전압 측정에 기반한 텐던진단장치를 이용하여 텐던의 손상을 진단함에 있...
86,거절,지하철 전동차내와 승강장 및 터널의 공기질(IAQ)을 콘트롤하는 인공지능 시스템공기...,1020190173722,20191224,이세라,,,,,"본 출원발명은 지하철 사령실, 전동차내, 승강장 및 터널내의 실내외공기질(IAQ 0..."
87,등록,인공지능을 활용한 자동화 병상 배정 방법(Method for Automated be...,1020220047582,20220418,주식회사 지티솔루션,최학현,주식회사 지티솔루션,,,본 발명은 입원과 퇴원으로 인한 병상관리를 인공지능(AI:artificial int...
88,등록,전파 측정 결과로부터 무인 항공기를 탐지하고 인공지능에 기반하여 무인 항공기를 식별...,1020220076281,20220622,주식회사 휴라,성병기,주식회사 휴라,,,"본 발명은 인공지능(artificial intelligence, AI)에 기반하여 ..."


In [ ]:
    # 페이지 목록 이동
    def page_num_move(self) :
        # 총 페이지 수 가져오기
        articles_info = self.driver.find_element(By.CSS_SELECTOR, 'p.articles').text
        total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())
        # print(f"Total pages: {total_pages}")

        for page_number in range(1, 12):
            # 페이지 번호를 클릭하여 페이지 변경
            if page_number != 1 and page_number % 10 != 0:
                page_link = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//span[@class='board_pager03']//a[text()='{page_number}']"))
                )
                page_link.click()
                time.sleep(3)  # 페이지가 로드될 때까지 대기
            elif page_number % 10 == 0:
                page_link = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[10]'))
                )
                page_link.click()
                time.sleep(3)  # 페이지가 로드될 때까지 대기

In [ ]:
articles_info = kipris.driver.find_element(By.CSS_SELECTOR, 'p.articles').text
total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())
for page_number in range(1, 12):
    # 페이지 번호를 클릭하여 페이지 변경
    if page_number != 1 and page_number % 10 != 0:
        page_link = WebDriverWait(kipris.driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[@class='board_pager03']//a[text()='{page_number}']"))
        )
        page_link.click()
        time.sleep(3)  # 페이지가 로드될 때까지 대기
    elif page_number % 10 == 0:
        page_link = WebDriverWait(kipris.driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[10]'))
        )
        page_link.click()
        time.sleep(3)  # 페이지가 로드될 때까지 대기

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF75B23EEA2+31554]
	(No symbol) [0x00007FF75B1B7ED9]
	(No symbol) [0x00007FF75B07872A]
	(No symbol) [0x00007FF75B0C8434]
	(No symbol) [0x00007FF75B0C853C]
	(No symbol) [0x00007FF75B10F6A7]
	(No symbol) [0x00007FF75B0ED06F]
	(No symbol) [0x00007FF75B10C977]
	(No symbol) [0x00007FF75B0ECDD3]
	(No symbol) [0x00007FF75B0BA33B]
	(No symbol) [0x00007FF75B0BAED1]
	GetHandleVerifier [0x00007FF75B548B1D+3217341]
	GetHandleVerifier [0x00007FF75B595AE3+3532675]
	GetHandleVerifier [0x00007FF75B58B0E0+3489152]
	GetHandleVerifier [0x00007FF75B2EE776+750614]
	(No symbol) [0x00007FF75B1C375F]
	(No symbol) [0x00007FF75B1BEB14]
	(No symbol) [0x00007FF75B1BECA2]
	(No symbol) [0x00007FF75B1AE16F]
	BaseThreadInitThunk [0x00007FFEB99D257D+29]
	RtlUserThreadStart [0x00007FFEB9FAAF28+40]


In [ ]:
articles_info = kipris.driver.find_element(By.CSS_SELECTOR, 'p.articles').text
total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())
articles_info

'Total 165,843 Articles (1 / 1,843 Pages)'

In [ ]:
# 상태 제목 출원번호 출원일자 출원인 대리인 최종권리자 우선권주장일자 국제출원일자 요약
# 상태 - status
# 제목 - title
# 출원번호 - application_number
# 출원일자 - application_date
# 출원인 - applicant
# 대리인 - representative
# 최종권리자 - final_owner
# 우선권주장일자 - priority_date
# 국제출원일자 - international_filing_date
# 요약 - summary

# 특허 목록 섹션
main_section = WebDriverWait(kipris.driver,20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#divMainArticle section.search_section'))
)

# 전체 목록 div
divs = main_section.find_elements(By.CSS_SELECTOR,'[id*="divViewSel"]') # id*= '문자'  -> 문자를 포함하는 아이디

status_list = []            # 상태
title_list = []             # 제목
idx_list = []               # 목록 번호
ap_num_list = []            # 출원번호
application_date_list = []  # 출원일자
applicant_list = []         # 출원인
representative_list = []    # 대리인


for idx,div in enumerate(divs) :
    h1_tag = div.find_element(By.CLASS_NAME,'stitle') # h1태그

    # 상태
    status = h1_tag.find_element(By.ID, 'iconStatus').text
    status_list.append(status)

    # 제목
    title = h1_tag.find_element(By.CSS_SELECTOR, 'div h1 a:nth-of-type(2)').text
    title_list.append(title)

    # 출원번호
    ap_num_tag = f'#divRealContent{idx} li:nth-child(2) > span.point01' # 출원번호 id -> divRealContent0, divRealContent1, divRealContent2...
    application_number = div.find_element(By.CSS_SELECTOR, ap_num_tag).text
    ap_num_list.append(application_number)

    # 출원일자
    ap_date_tag = f'#divRealContent{idx} li:nth-child(3)'
    ap_date_text = div.find_element(By.CSS_SELECTOR, ap_date_tag).text # 출원일자 : 2020.09.25
    application_date = re.sub(r'[^\d]','', div.find_element(By.CSS_SELECTOR, ap_date_tag).text) # 숫자만 추출
    application_date_list.append(application_date)

    # 출원인
    applicant_tag = f'#divRealContent{idx} ul.search_info_list li:nth-child(4) a'
    applicant = div.find_element(By.CSS_SELECTOR, applicant_tag).text
    applicant_list = applicant

    # 대리인
    representative_tag = f'#divRealContent{idx} ul.search_info_list li:nth-child(10) a'
    try:
        # 웹 요소에서 텍스트 추출
        representative = div.find_element(By.CSS_SELECTOR, representative_tag).text
    except Exception :
        # 요소가 존재하지 않는 경우 빈 문자열로 설정
        representative = ''

    # 결과를 리스트 변수에 저장
    representative_list = representative

    # for문 끝

# 상태 - status
# 제목 - title
# 출원번호 - application_number
# 출원일자 - application_date
# 출원인 - applicant
# 대리인 - representative
# 최종권리자 - final_owner
# 우선권주장일자 - priority_date
# 국제출원일자 - international_filing_date
# 요약 - summary

df = pd.DataFrame({
    'status'                :   status_list,
    'title'                 :   title_list,
    'application_number'    :   ap_num_list,
    'application_date'      :   application_date_list,
    'applicant'             :   applicant_list,
    'representative'        :representative_list
})

df

,status,title,application_number,application_date,applicant,representative
0,등록,"갱신 영역 학습을 통한 인공지능 기반의 영상 분석 시스템 및 방법과, 이를 위한 컴...",1020200124561,20200925,주식회사 인피닉,정두승
1,등록,"인공지능(Artificial Intelligence, AI) 알고리즘 수행 방법, ...",1020200165605,20201201,주식회사 인피닉,정두승
2,취하,인공지능 돌봄 서비스 시스템(An AI Care Service System),1020200071313,20200612,주식회사 인피닉,정두승
3,공개,인공지능 학습용 이미지 인식 데이터 품질관리를 활용한 자동화된 머신러닝 시스템(Au...,1020220140941,20221028,주식회사 인피닉,정두승
4,등록,"머신러닝 기반 인공지능 모델 학습, 개발, 배포 및 운영 시스템과 이를 이용한 서비...",1020200164624,20201130,주식회사 인피닉,정두승
...,...,...,...,...,...,...
85,등록,"텐던진단장치를 이용한 텐던 손상진단을 위한 인공지능 학습방법과 인공지능 학습시스템,...",1020220169250,20221207,주식회사 인피닉,정두승
86,거절,지하철 전동차내와 승강장 및 터널의 공기질(IAQ)을 콘트롤하는 인공지능 시스템공기...,1020190173722,20191224,주식회사 인피닉,정두승
87,등록,인공지능을 활용한 자동화 병상 배정 방법(Method for Automated be...,1020220047582,20220418,주식회사 인피닉,정두승
88,등록,전파 측정 결과로부터 무인 항공기를 탐지하고 인공지능에 기반하여 무인 항공기를 식별...,1020220076281,20220622,주식회사 인피닉,정두승


In [ ]:
kipris.driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="f0274d30626f729344eed50dd67649ae")>

In [ ]:
df.iloc[35]

status                                                             등록
title               인공지능(AI) 기반 콘텐츠 융합 서비스 시스템 및 방법(CONTENTS MERGI...
ap_num                                                  1020190143352
application_date                                             20191111
applicant                                                 주식회사 아임클라우드
representative                                                    김동진
final_owner                                                 주식회사 시니스트
priority_date                                                20190530
Name: 35, dtype: object

In [ ]:
# 특허 목록 섹션
main_section = WebDriverWait(kipris.driver,20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#divMainArticle section.search_section'))
)

# 전체 목록 div
divs = main_section.find_elements(By.CSS_SELECTOR,'[id*="divViewSel"]') # id*= '문자'  -> 문자를 포함하는 아이디

status_list = []            # 상태
title_list = []             # 제목
idx_list = []               # 목록 번호
ap_num_list = []            # 출원번호
application_date_list = []  # 출원일자
applicant_list = []         # 출원인
representative_list = []    # 대리인


for idx,div in enumerate(divs) :
    h1_tag = div.find_element(By.CLASS_NAME,'stitle') # h1태그

    # 상태
    status = h1_tag.find_element(By.ID, 'iconStatus').text
    status_list.append(type)

    # 제목
    title = h1_tag.find_element(By.CSS_SELECTOR, 'div h1 a:nth-of-type(2)').text
    title_list.append(title)

In [ ]:
# 로그인
def login(self) :
    print('login() start')
    login = self.driver.find_element(By.ID, 'btnLogin')
    login.click()

    print(self.driver.window_handles)

    # 로그인 팝업창 선택
    driver.switch_to.window(self.driver.window_handles[1])

    # 로그인 ID
    login_id = self.driver.find_element(By.ID, 'login_id')
    login_id.send_keys('boseon169@gmail.com')

    # 로그인 pw
    login_pw = self.driver.find_element(By.ID, 'login_pw')
    login_pw.send_keys('!@1234boseon')

    # 로그인 버튼
    btn_login = self.driver.find_element(By.ID, 'btn_login')
    btn_login.click()

    pop_close()

    print('login() end')


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time
import re
import pandas as pd

# 사이트 열기
driver = webdriver.Chrome()
url = 'http://www.kipris.or.kr/khome/main.jsp'

driver.get(url)

# 팝업창 닫기
pop_close(driver)

# 전체 페이지
driver.maximize_window()

# 로그인 실행
login(driver)

# 특허·실용신안 페이지 이동
#patent_move(driver)

In [ ]:
# 팝업창 닫기
def pop_close(self, driver) :
    print('pop_close() start')

    # 팝업이 있는지 10초동안 확인
    wait = WebDriverWait(driver, 10)
    wait.until(lambda d: len(driver.window_handles) > 1)

    print('페이지 갯수 : ' , len(driver.window_handles))

    # 팝업이 있을 경우 팝업 닫기
    if len(driver.window_handles) > 1 :
        total_site = driver.window_handles # 열린 사이트 갯수 (메인페이지 + 팝업창)
        main_page= driver.current_window_handle # 메인 페이지 코드

        print(f'총 페이지 : {total_site}')
        print(f'메인 페이지 코드 : {main_page}')

        for page in total_site :
            # 열린 사이트들에서 메인 페이지를 제외한 창 닫기
            if page != main_page :
                driver.switch_to.window(page) # 현재 선택된 창을 팝업창으로 변경
                driver.close()

        # 팝업창을 다 닫은 후 메인페이지를 다시 선택
        driver.switch_to.window(main_page)

    print('pop_close() end')


In [ ]:
# 로그인
def login(self, driver) :
    print('login() start')
    login = driver.find_element(By.ID, 'btnLogin')
    login.click()

    print(driver.window_handles)

    # 로그인 팝업창 선택
    driver.switch_to.window(driver.window_handles[1])

    # 로그인 ID
    login_id = driver.find_element(By.ID, 'login_id')
    login_id.send_keys('boseon169@gmail.com')

    # 로그인 pw
    login_pw = driver.find_element(By.ID, 'login_pw')
    login_pw.send_keys('!@1234boseon')

    # 로그인 버튼
    btn_login = driver.find_element(By.ID, 'btn_login')
    btn_login.click()

    pop_close(driver)

    print('login() end')


In [ ]:
def patent_move(driver) :
    print('patent_move start')

    # 특허·실용신안 태그 가져오기
    patent_tag = WebDriverWait(driver,3).until(
        EC.presence_of_element_located((By.CSS_SELECTOR,'#gnb ul.cbp-hrsub-inner.cbp-hrsub-inner.depth2 li:nth-child(1) a'))
    )

    # 클릭하여 페이지 이동
    patent_tag.click()

    # 페이지 이동 잠시 대기
    time.sleep(3)

    # 검색창
    input_tag = driver.find_element(By.ID,'queryText')
    input_tag.send_keys('("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"머신러닝"+"machine learning")*AD=[20190624~20240624]')
    input_tag.send_keys(Keys.ENTER)

    time.sleep(3)

    select = Select(driver.find_element(By.ID,'opt28'))
    select.select_by_value('90')

    time.sleep(3)

    select_a = driver.find_element(By.CSS_SELECTOR,'#pageSel a')
    select_a.click()

    print('patent_move end')

In [ ]:
# 특허 데이터 가져오기
main_section = WebDriverWait(driver,20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#divMainArticle section.search_section'))
)

# 전체 목록 div
divs = main_section.find_elements(By.CSS_SELECTOR,'[id*="divViewSel"]')

type_list = []
title_list = []     # 제목
idx_list = []       # 목록 번호
ap_num_list = []    # 출원번호
date_list = []      # 출원일자


for div in divs :
    title = div.find_element(By.CLASS_NAME,'stitle').text.split()

    type_list.append(title[0])

    # 제목
    title_list.append(title[2]) # 제목

    # 리스트 번호
    # idx = re.sub(r'[^\d]','',title.split()[1])
    # print(idx)

    # 특허별 div
    list_div = div.find_element(By.CLASS_NAME,'mainlist_topinfo')

    # 출원번호 / 일자
    num_date = list_div.find_element(By.CSS_SELECTOR, 'li.left_width:nth-of-type(2) span.point01 a').text

    # 출원번호
    ap_num_list.append(num_date.split()[0])

    # 일자
    date_list.append(re.sub(r'[^\d\.]', '', num_date.split()[1]))

    # ipc
    # ipcs = list_div.find_elements(By.CSS_SELECTOR,'li:nth-of-type(1) span.point01')
    # for ipc in ipcs :
    #     print(ipc.text)



df = pd.DataFrame({
    'Type'      : type_list,
    'Title'     : title_list,
    'ApNum'     : ap_num_list,
    'ApDate'    : date_list

})

df

In [ ]:
# 페이지 이동
page_div = driver.find_element(By.CSS_SELECTOR,'section.search_section div.float_left p.articles')


total_page = page_div.find_element(By.CLASS_NAME,'total').text
total_page = int(re.sub(r'[^\d]', '' , total_page))

print('총페이지 : ' , total_page)


current_page = int(page_div.find_element(By.CLASS_NAME,'current').text)
print('현재 페이지 : ' , current_page)

# 다음 페이지 이동 버튼
next_btn = driver.find_element(By.CSS_SELECTOR,'section.search_section div.float_right a.next')

for i in range(12) :
    # 다음페이지 이동
    next_btn.click()

    next_page = current_page + 1

    WebDriverWait(driver,10).until(
        #EC.presence_of_element_located((By.CSS_SELECTOR, 'section.search_section div.float_left p.articles span.current'))
        EC.text_to_be_present_in_element((By.CSS_SELECTOR, 'section.search_section div.float_left p.articles span.current'),str(next_page))
    )

    # 현재 페이지 번호 수정
    current_page = next_page

In [ ]:
    def page_num_move(self) :
        # # 총 페이지 수 가져오기
        # articles_info = self.driver.find_element(By.CSS_SELECTOR, 'p.articles').text
        # total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())

        # # print(f"Total pages: {total_pages}")

        # # 페이지 번호 목록
        # list_text = self.driver.find_element(By.CSS_SELECTOR, '#divMainArticle div.float_right span').text

        # # 이전, 다음 뺴고 숫자만 리스트 형식으로 추출
        # list_num = re.findall(r'\d+', list_text)

        # sPage = list_num[0]     # 첫 페이지
        # ePage = list_num[-1]    # 마지막 페이지

        # for page_number in range(1, total_pages):
        #     current_page = self.driver.find_element(By.CSS_SELECTOR, '#divMainArticle div.float_left span.current').text # 현재 페이지
        #     print('현재페이지 : ' , current_page)

        #     # 현재페이지 번호가 총페이지 번호랑 같지 않을 경우에만 진행
        #     if current_page != total_pages :
        #         # 페이지 번호를 클릭하여 페이지 변경
        #         if current_page != ePage :
        #             page_link = WebDriverWait(self.driver, 10).until(
        #                 #EC.element_to_be_clickable((By.XPATH, f"//span[@class='board_pager03']//a[text()='{page_number}']"))
        #                 EC.element_to_be_clickable((By.XPATH, f"//*[@id='divMainArticle']/form/section/div[2]/span/a['{page_number}']"))
        #             )
        #             page_link.click()
        #             time.sleep(3)  # 페이지가 로드될 때까지 대기

        #         # 현재 페이지번호와 마지막 목록 페이지 번호가 같을 경우 다음 버튼 클릭
        #         elif current_page == ePage : # 다음 버튼
        #             page_link = WebDriverWait(self.driver, 10).until(
        #                 EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[10]'))
        #             )
        #             page_link.click()
        #             time.sleep(3)  # 페이지가 로드될 때까지 대기